In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

In [12]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3806068386&distance=25&f_AL=true&f_JT=F&f_WT=2&geoId=101318387&keywords=operations%20analyst&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R")

scroll_pause_time = 2 
scroll_height = 0

time.sleep(100)

while True:

    #Current scroll height
    prev_scroll_height = scroll_height
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    # scroll down the page
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")

    # wait 
    time.sleep(scroll_pause_time)

    # no more new listing are loaded, break the loop
    if scroll_height == prev_scroll_height:
        break

jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')


with open('job_listing.csv','w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Job Title', 'Company Name', 'Address', 'Job Link']) #header row
    for listing in jobs_listing:
        job_title_element = listing.find_element(By.TAG_NAME, 'strong')
        job_title = job_title_element.text

        company_element = listing.find_element(By.CLASS_NAME, 'job-card-container__primary-description ')
        company = company_element.text

        address_element = listing.find_element(By.CLASS_NAME, 'job-card-container__metadata-item ')
        address = address_element.text

        # try:
        #     salary_element = listing.find_element(By.CLASS_NAME, 'job-search-card__salary-info')
        #     salary = salary_element.text
        # except:
        #     salary = "0"
        

            # Check if job link is available
        try:
            link_element = listing.find_element(By.CLASS_NAME, 'disabled ember-view job-card-container__link job-card-list__title')
            job_link = link_element.get_attribute('href')
        except:
            job_link = "N/A"


        print("Job Title:", job_title)
        print("Company:", company)
        print("Address:", address)
        #print("Salary:", salary)
        print("Link:", job_link)
        print("-" * 50)

        csv_writer.writerow([job_title, company, address,job_link])

#driver.quit()

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*[@id="main-content"]/section[2]/ul/ because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//*[@id="main-content"]/section[2]/ul/' is not a valid XPath expression.
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6222C7012+3522402]
	(No symbol) [0x00007FF621EE8352]
	(No symbol) [0x00007FF621D95ABB]
	(No symbol) [0x00007FF621D9AD09]
	(No symbol) [0x00007FF621D9D110]
	(No symbol) [0x00007FF621D9D1D0]
	(No symbol) [0x00007FF621DDBB7C]
	(No symbol) [0x00007FF621DDC08C]
	(No symbol) [0x00007FF621E1E467]
	(No symbol) [0x00007FF621DFF09F]
	(No symbol) [0x00007FF621E1BDA3]
	(No symbol) [0x00007FF621DFEE03]
	(No symbol) [0x00007FF621DCF4D4]
	(No symbol) [0x00007FF62

In [4]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Path to ChromeDriver executable
chromedriver_path = r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Initialize ChromeDriver
s = Service(chromedriver_path)
driver = webdriver.Chrome(service=s)

# Read CSV file containing job details and URLs
df = pd.read_csv('job_listing.csv')

# Function to extract expected salary from URL
def extract_expected_salary(url):
    max_attempts = 3
    for attempt in range(max_attempts):
        time.sleep(3)
        try:
            driver.get(url)
            expected_salary_element = driver.find_element(By.CLASS_NAME, 'salary.compensation__salary')
            expected_salary = expected_salary_element.text
            return expected_salary
        except NoSuchElementException:
            print(f"Attempt {attempt + 1}: Expected salary not found. Retrying...")
            time.sleep(2)  # Pause for a moment before retrying

    return "N/A"

# Extract expected salary for each job listing
df['Expected Salary'] = df['Job Link'].apply(extract_expected_salary)

# Save updated job details to CSV
df.to_csv('job_details_with_salary.csv', index=False)

# Close the ChromeDriver session
driver.quit()

C:\Users\Hebert Palmera\AppData\Local\Temp\ipykernel_3340\2621584761.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
